In [39]:
import rhinoinside
rhinoinside.load()
import System
from Rhino.Geometry import Point3d, Plane, Brep, Vector3d
from Rhino.Geometry import Surface as RhinoSurface
import Rhino.Geometry as rg
from Rhino.Geometry import Polyline
from Rhino.Collections import Point3dList
from Rhino.Geometry import PolylineCurve

In [40]:
def create_surface(x, y, width, height):
    # Initialiser les points pour la polyline
    points = Point3dList()
    points.Add(x, y, 0)
    points.Add(x + width, y, 0),
    points.Add(x + width, y + height, 0),
    points.Add(x, y + height, 0),
    points.Add(x, y, 0)
    

    # Créer une polyline
    polyline = PolylineCurve(points)
    
    # Vérifier si la polyline est fermée
    if polyline.IsClosed:
        # Créer une extrusion de la polyline
        direction = Vector3d(0, 0, 0.001)  # Direction d'extrusion (minimale pour simuler une surface)
        surface = RhinoSurface.CreateExtrusion(polyline, direction)  # Extrusion minime
        return surface
    else:
        return None


In [41]:
def create_grid_surfaces(rows, cols, width, height, angle):
    grid_surfaces = []
    for i in range(rows):
        for j in range(cols):
            # Position de la surface dans la grille
            x = j * (width)  # Espacement de 0 entre les colonnes
            y = i * (height + 10)  # Espacement de 10 entre les lignes
            surface = create_surface(x, y, width, height)
            # Rotation de la surface
            center = Point3d(x + width / 2, y + height / 2, 0)
            rotation_axis = Vector3d(0, 1, 0)  # Axe Y pour la rotation
            surface.Rotate(System.Math.PI / 180 * angle, rotation_axis, center)
            grid_surfaces.append(surface)
    return grid_surfaces


In [42]:
def join_surfaces_to_brep(surfaces):
    brep = rg.Brep()
    for surface in surfaces:
        # Convertir la surface en Brep et ajouter au Brep principal
        brep_part = surface.ToBrep()
        brep.Append(brep_part)
    return brep

# Paramètres de la grille
rows = 20  # Nombre de lignes
cols = 40  # Nombre de colonnes
width = 5  # Largeur de chaque surface
height = 1.135  # Hauteur de chaque surface
angle = 30  # Angle d'inclinaison en degrés

# Création de la grille et assemblage en un seul Brep
surfaces = create_grid_surfaces(rows, cols, width, height, angle)
final_brep = join_surfaces_to_brep(surfaces)


In [43]:
from Rhino.FileIO import File3dm

def export_brep_to_file(brep, filename):
    file = File3dm()
    file.Objects.AddBrep(brep)
    file.Write(filename, 7)  # Version 7 pour Rhino 7

# Exporter le Brep comme un fichier 3DM
export_brep_to_file(final_brep, "output.3dm")